In [2]:
using Pkg; Pkg.activate(".."); Pkg.instantiate()

using OceanTurb

import OceanTurb: Diffusion              

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


┌ Info: Precompiling OceanTurb [d848d694-35fd-11e9-1c91-9995598ac29e]
└ @ Base loading.jl:1186


In [ ]:
model = Diffusion.Model(nz=100, Lz=1.0, κ=0.01)
stepper = Timestepper(:ForwardEuler, model.solution)